# Projeto 1 - Ciência dos Dados

Nome: Vitor Fortes Giuliano Riccetti 

Nome: Rodrigo Paoliello de Medeiros

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'Chuteira_Puma.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Chuteira_Puma.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
# Faz a leitura do excel 
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

train["Relevante"]= train["Relevante"].astype(str)
train["Irrelevante"]= train["Irrelevante"].astype(str)



In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)



,Teste,classificacao
0,@gathlos de níver atrasado quero uma chuteira ...,Relevante
1,@neymarjrdepre @neymarjr @puma essa chuteira é...,Relevante
2,era só essa chuteira da puma na minha vida,Relevante
3,@manassescruz1 @juancrf19 @futebol_info @puma_...,Relevante
4,só a chuteira da puma na minha vida mesmo.,Relevante


In [5]:
# dividimos em duas categorias relevante e irrelevante:

Relevantes = ''.join(train.Relevante)

Irrelevantes = ' '.join(train.Irrelevante)


In [6]:
# Função para limpar a string
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    text = re.sub(r'@\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'_', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\n', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'kk\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'http\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', ' ', text, flags=re.MULTILINE)
    punctuation = '[!-.:?;_“”]' # Note que os sinais [] são delimitadores de um conjunto.

    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

emoji_pat = '[\U0001F300-\U0001F64F\U0001F680-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]'
shrink_whitespace_reg = re.compile(r'\s{2,}')

def clean_text(text_subbed):
    reg = re.compile(r'({})'.format(emoji_pat)) # line a
    result = reg.sub(lambda x: ' {} '.format(x.group(1)) if x.group(1) else ' ', text_subbed)
    return shrink_whitespace_reg.sub(' ', result)




In [7]:
# Retira as pontuações e os links

Lista_R_clean = cleanup(Relevantes.lower())
Lista_R_clean = clean_text(Lista_R_clean)



In [8]:
# Retira as pontuações e os links

Lista_IR_clean = cleanup(Irrelevantes.lower())
Lista_IR_clean = clean_text(Lista_IR_clean)




In [9]:
# Serie de palavras Relevantes
Palavras_relevantes = pd.Series(Lista_R_clean.split())

len(Palavras_relevantes)

1759

In [10]:
# Serie de palavras Irrelevantes
Palavras_irrelevantes = pd.Series(Lista_IR_clean.split())

len(Palavras_irrelevantes)

2105

In [11]:
# Tabela absoluta dos Relevantes.
tabela_absoluta_relevantes = Palavras_relevantes.value_counts(ascending=False)




In [12]:
# Tabela absoluta dos Irrelevantes.
tabela_absoluta_irrelevantes = Palavras_irrelevantes.value_counts(ascending=False)




In [13]:
# Frequencia relativa dos Relevantes 

tabela_relativa_relevantes = Palavras_relevantes.value_counts(True)



In [14]:
tabela_relativa_relevantes.sum()

1.0

In [15]:
# Frequencia relativa dos Irrelevantes

tabela_relativa_irrelevantes = Palavras_irrelevantes.value_counts(True)



In [16]:
tabela_relativa_irrelevantes.sum()

0.9999999999999999

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [17]:
total = len(Palavras_relevantes) + len(Palavras_irrelevantes)
total_relevantes = len(Palavras_relevantes)/total
total_irrelevantes = len(Palavras_irrelevantes)/total
print(total_relevantes)
print(total_irrelevantes)

0.4552277432712215
0.5447722567287785


In [18]:
#palavras que nos selecionamos 

lista_palavras = ['chuteira','bonita','feia','quero','comprar','não','gostei','ruim','boa','puma','nike','adidas','melhor','neymar','estranho']


In [19]:
#probabilidade das palavras estarem como relevantes 

chuteira_r = (tabela_absoluta_relevantes['chuteira'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
bonita_r = (tabela_absoluta_relevantes['bonita'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
feia_r = (tabela_absoluta_relevantes['feia'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
quero_r = (tabela_absoluta_relevantes['quero'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
comprar_r = (tabela_absoluta_relevantes['comprar'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
nao_r = (tabela_absoluta_relevantes['não'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
gostei_r = (tabela_absoluta_relevantes['gostei'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
ruim_r = (tabela_absoluta_relevantes['ruim'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
boa_r = (tabela_absoluta_relevantes['boa'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
puma_r = (tabela_absoluta_relevantes['puma'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
nike_r = (tabela_absoluta_relevantes['nike'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
adidas_r = (tabela_absoluta_relevantes['adidas'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
melhor_r = (tabela_absoluta_relevantes['melhor'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
neymar_r = (tabela_absoluta_relevantes['neymar'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
estranho_r = (tabela_absoluta_relevantes['estranho'] + 1)/(len(Palavras_relevantes) + len(lista_palavras))

In [20]:
#probabilidade das palavras estarem como irrelevantes 

chuteira_ir = (tabela_absoluta_irrelevantes['chuteira'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
bonita_ir = (tabela_absoluta_irrelevantes['bonita'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
feia_ir = (tabela_absoluta_irrelevantes['feia'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
quero_ir = (tabela_absoluta_irrelevantes['quero'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
comprar_ir = (tabela_absoluta_irrelevantes['comprar'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
nao_ir = (tabela_absoluta_irrelevantes['não'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
gostei_ir = (tabela_absoluta_irrelevantes['gostei'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
ruim_ir = (0 + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
boa_ir = (tabela_absoluta_irrelevantes['boa'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
puma_ir = (tabela_absoluta_irrelevantes['puma'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
nike_ir = (tabela_absoluta_irrelevantes['nike'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
adidas_ir = (tabela_absoluta_irrelevantes['adidas'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
melhor_ir = (tabela_absoluta_irrelevantes['melhor'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
neymar_ir = (tabela_absoluta_irrelevantes['neymar'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
estranho_ir = (tabela_absoluta_irrelevantes['estranho'] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))

In [21]:
P_chuteira_bonita_r = chuteira_r * bonita_r * total_relevantes
P_chuteira_bonita_r

0.0001397328822055775

In [22]:
P_chuteira_bonita_ir = chuteira_ir * bonita_ir * total_irrelevantes
P_chuteira_bonita_ir

3.757551610580307e-05

In [23]:
P_chuteira_feia_r = chuteira_r * feia_r * total_relevantes
P_chuteira_feia_r

0.000279465764411155

In [24]:
P_chuteira_feia_ir = chuteira_ir * feia_ir * total_irrelevantes
P_chuteira_feia_ir

3.757551610580307e-05

In [25]:
P_quero_comprar_chuteira_r = quero_r * comprar_r * chuteira_r * total_relevantes
P_quero_comprar_chuteira_r

5.73874777942056e-07

In [26]:
P_quero_comprar_chuteira_ir = quero_ir * comprar_ir * chuteira_ir * total_irrelevantes
P_quero_comprar_chuteira_ir

1.0634580029944266e-07

In [27]:
P_nike_melhor_r = nike_r * melhor_r * total_relevantes
P_nike_melhor_r

4.773483553606684e-06

In [28]:
P_nike_melhor_ir = nike_ir * melhor_ir * total_irrelevantes
P_nike_melhor_ir

1.0181752751249865e-05

In [29]:
P_adidas_melhor_r = adidas_r * melhor_r * total_relevantes
P_adidas_melhor_r

1.3018591509836412e-06

In [30]:
P_adidas_melhor_ir = adidas_ir * melhor_ir * total_irrelevantes
P_adidas_melhor_ir

1.939381476428546e-06

In [31]:
P_neymar_puma_estranho_r = neymar_r * puma_r * estranho_r * total_relevantes
P_neymar_puma_estranho_r

4.0769734153314575e-07

In [32]:
P_neymar_puma_estranho_ir = neymar_ir * puma_ir * estranho_ir * total_irrelevantes
P_neymar_puma_estranho_ir

2.2291451946638017e-06

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [40]:


frase_testes = []
classificador = []

for tweet in test.Teste:
    y = cleanup(tweet.lower())
    tweets = clean_text(y)
    r = 1
    i = 1
 
    for x in tweets:
        
        if x not in tabela_absoluta_relevantes:
            r = r * (0 + 1)/(len(Palavras_relevantes) + len(lista_palavras))
        else:
            r = r *  (tabela_absoluta_relevantes[x] + 1)/(len(Palavras_relevantes) + len(lista_palavras))
        if x not in tabela_absoluta_irrelevantes:
            i = i * (0 + 1)/(len(Palavras_irrelevantes) + len(lista_palavras))
        else:
            i = i * (tabela_absoluta_irrelevantes[x] + 1)/(len(Palavras_irrelevantes) + len(lista_palavras)) 
            
    R = total_relevantes * r
    I = total_irrelevantes * i
    if R > I: 
        classificador.append("Relevante")
    else:
        classificador.append("Irrelevante")


        
print(classificador)


['Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Irrelevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevante', 'Relevan

In [41]:
test['classificador'] = classificador
test

,Teste,classificacao,classificador
0,@gathlos de níver atrasado quero uma chuteira ...,Relevante,Relevante
1,@neymarjrdepre @neymarjr @puma essa chuteira é...,Relevante,Relevante
2,era só essa chuteira da puma na minha vida,Relevante,Relevante
3,@manassescruz1 @juancrf19 @futebol_info @puma_...,Relevante,Relevante
4,só a chuteira da puma na minha vida mesmo.,Relevante,Relevante
...,...,...,...
294,no aguardo da proposta milionária na dm pra co...,Irrelevante,Relevante
295,"puma, chuteira preta hmm\nfedeu a hexa https:/...",Irrelevante,Relevante
296,serase o @neymarjr vai ressarcir meu dinheiro ...,Irrelevante,Relevante
297,só jogo lixo às 16hrs nessa rodada do br.\n\né...,Irrelevante,Irrelevante


In [43]:
pd.crosstab(test['classificacao'], test['classificador'], normalize='columns')

classificador,Irrelevante,Relevante
classificacao,,
Irrelevante,0.685185,0.497959
Relevante,0.314815,0.502041


___
### Concluindo

In [ ]:
acertou 68% dos irr
acertou 50,2% dos re

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**